## install moveit_commander

```
sudo apt-get install ros-"DISTRO"-moveit-commander
pip install pyassimp
```

## reference

https://ros-planning.github.io/moveit_tutorials/doc/move_group_python_interface/move_group_python_interface_tutorial.html

## Before Running Moveit 

```
# run gazebo
roslaunch ur_gazebo ur5.launch
# run moveit
roslaunch ur5_moveit_config ur5_moveit_planning_execution.launch sim:=true limited:=true
# run rviz
roslaunch ur5_moveit_config moveit_rviz.launch config:=true
```


In [3]:
import os
ROS_DISTRO = os.environ['ROS_DISTRO']

In [4]:
from __future__ import print_function
from six.moves import input

import sys
ros_python_path = '/opt/ros/{}/lib/python2.7/dist-packages'.format(ROS_DISTRO)
if not ros_python_path in sys.path:
    sys.path.append(ros_python_path)

import rospy
import copy
import numpy as np

In [5]:
from moveit_commander.conversions import pose_to_list
from std_msgs.msg import String
import geometry_msgs.msg
import moveit_commander
import moveit_msgs.msg
from math import pi

## initialize moveit_commander

In [6]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_python_interface_tutorial', anonymous=True)

In [7]:
# Provides information such as the robot’s kinematic model and the robot’s current joint states
robot = moveit_commander.RobotCommander()

In [8]:
robot.get_link_names()

['base_link',
 'base',
 'base_link_inertia',
 'shoulder_link',
 'upper_arm_link',
 'forearm_link',
 'wrist_1_link',
 'wrist_2_link',
 'wrist_3_link',
 'flange',
 'tool0']

In [18]:
robot.get_link('tool0').pose().pose.position

x: 0.0281699409803
y: -0.214232267321
z: 0.600548908959

In [20]:
robot.get_link('tool0').pose().pose.orientation

x: 0.958086076905
y: -0.285740193299
z: -0.00504796037302
w: -0.0199531769588

In [41]:
robot.get_link('tool0').pose().pose.position

x: 0.009212875433
y: -0.25998312214
z: 0.511016860373

In [42]:
robot.get_link('tool0').pose().pose.orientation

x: 0.775775995589
y: -0.630675178119
z: -0.0123380940415
w: -0.0163766848868

In [44]:
robot.get_link('tool0').pose().pose.position

x: -0.0288125235908
y: -0.244735781865
z: 0.522612435925

In [45]:
robot.get_link('tool0').pose().pose.orientation

x: 0.925123692511
y: -0.270638414597
z: -0.262335350605
w: -0.0456197974789

In [10]:
robot.get_current_state()

joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_link"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint]
  position: [-1.9041474501239222, -1.9014700094806116, 1.2771167755126953, -0.8735716978656214, 4.636214733123779, -3.311577622090475]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_link"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

In [11]:
#'''
print(robot.get_group_names())
print(robot.get_current_state())
print(robot.get_joint_names())
#'''

['endeffector', 'manipulator']
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_link"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint]
  position: [-1.9041474501239222, -1.901421848927633, 1.2771406173706055, -0.8735960165606897, 4.636214733123779, -3.3115535418139856]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_link"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False
['fixed_base', 'base_link-base_fixed_joint', 'base_link-base_link_inertia', 'shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint', 'wrist_3-flange', 'flange-tool0']


In [12]:
# provides a remote interface for getting, setting, and updating the robot’s internal understanding of the surrounding world
# 플래닝할때, 주변 장애물을 반영할 수 있게, 아래의 object를 통해 환경에 장애물을 추가할 수 있다.
scene = moveit_commander.PlanningSceneInterface()

In [13]:
# plan 한 trajectory를 rviz화면에 시뮬레이션해줌.
# "roslaunch ur5_moveit_config moveit_rviz.launch config:=true" 을 통해 미리 rviz를 띄어놔야 함.
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path',
                                               moveit_msgs.msg.DisplayTrajectory,
                                               queue_size=20)

In [14]:
# move_group 객체를 통해 각 joint(move_element)의 desired states를 지정할 수 있고, moveit은 이 값을 바탕으로 planning을 해줌.
group_name = "manipulator"
move_group = moveit_commander.MoveGroupCommander(group_name)

In [15]:
# We can get the name of the reference frame for this robot:
planning_frame = move_group.get_planning_frame()
print("============ Planning frame: %s" % planning_frame)

# We can also print the name of the end-effector link for this group:
eef_link = move_group.get_end_effector_link()
print("============ End effector link: %s" % eef_link)

# We can get a list of all the groups in the robot:
group_names = robot.get_group_names()
print("============ Available Planning Groups:", robot.get_group_names())

# Sometimes for debugging it is useful to print the entire state of the
# robot:
print("============ Printing robot state")
print(robot.get_current_state())
print("")

============ Planning frame: base_link
============ End effector link: tool0
============ Available Planning Groups: ['endeffector', 'manipulator']
============ Printing robot state
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_link"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint]
  position: [-1.9040992895709437, -1.9014337698565882, 1.2771406173706055, -0.8735716978656214, 4.636226654052734, -3.3115535418139856]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_link"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False



In [16]:
def move_to_joints(move_group, joint_goal):
    move_group.go(joint_goal, wait=True)
    move_group.stop()
    
    current_joints = move_group.get_current_joint_values()
    return current_joints
    
def get_pose(move_group):
    return move_group.get_current_pose().pose

In [17]:
init_joints = [-1.9631989637957972, -2.113516632710592, 1.476273536682129, -0.894970719014303, -1.5849650541888636, -2.9540748417318197]

move_to_joints(move_group, init_joints)

[-1.963294808064596,
 -2.1135643164264124,
 1.4763455390930176,
 -0.8948991934405726,
 -1.5849407354937952,
 -2.9542420546161097]

In [28]:
# path planning
delta_x = 0.01
delta_y = 0.015
delta_z = 0.015

waypoints = []
wpose = move_group.get_current_pose().pose

wpose.position.x += delta_x
waypoints.append(copy.deepcopy(wpose))
wpose.position.y += delta_y
waypoints.append(copy.deepcopy(wpose))
wpose.position.z += delta_z
waypoints.append(copy.deepcopy(wpose))

wpose.position.x -= delta_x
wpose.position.y -= delta_y
wpose.position.z -= delta_z
waypoints.append(copy.deepcopy(wpose))

(plan, fraction) = move_group.compute_cartesian_path(
                                   waypoints,   # waypoints to follow
                                   0.01,        # eef_step
                                   0.0)         # jump_threshold

display_trajectory = moveit_msgs.msg.DisplayTrajectory()
display_trajectory.trajectory_start = robot.get_current_state()
display_trajectory.trajectory.append(plan)

display_trajectory_publisher.publish(display_trajectory);

move_group.execute(plan, wait=True)

True

## robotiq finger

In [29]:
# for gripper
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
import actionlib

In [30]:
action_name = rospy.get_param('~action_name'base_link, 'command_robotiq_action')
print(action_name)

robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()
print("Client test: Starting sending goals")

command_robotiq_action
Client test: Starting sending goals


In [31]:
## Manually set all the parameters of the gripper goal state.
######################################################################################

def open_grasp(robotiq_client):
    goal = CommandRobotiqGripperGoal()
    goal.emergency_release = False
    goal.stop = False
    goal.position = 0.8
    goal.speed = 0.1
    goal.force = 1.0
    robotiq_client.send_goal(goal)
    robotiq_client.wait_for_result()

def close_grasp(robotiq_client):
    goal = CommandRobotiqGripperGoal()
    goal.emergency_release = False
    goal.stop = False
    goal.position = 0.0
    goal.speed = 0.1
    goal.force = 5.0
    robotiq_client.send_goal(goal)
    robotiq_client.wait_for_result()

In [46]:
move_group.set_max_velocity_scaling_factor = 0.001
joint_goal = move_group.get_current_joint_values()
joint_goal[0] += 0
joint_goal[1] += 0
joint_goal[2] += 0
joint_goal[3] += 0
joint_goal[4] += np.pi/6
joint_goal[5] += 0

# The go command can be called with joint values, poses, or without any
# parameters if you have already set the pose or joint target for the group
move_group.go(joint_goal, wait=True)

# Calling ``stop()`` ensures that there is no residual movement
move_group.stop()